In [ ]:
!pip install slack_bolt slack_sdk langgraph langchain google-generativeai langchain_google_genai python-dotenv notion-client


In [ ]:
%%writefile .env
SLACK_BOT_TOKEN=token
SLACK_SIGNING_SECRET=secret
GOOGLE_API_KEY=API
NOTION_API_KEY=API
SLACK_APP_TOKEN=token

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load from .env file

# Slack credentials
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
SLACK_SIGNING_SECRET = os.getenv("SLACK_SIGNING_SECRET")

# Google Gemini API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Notion API key
NOTION_API_KEY = os.getenv("NOTION_API_KEY")


SOCKET_MODE_APP_TOKEN = os.getenv("SLACK_APP_TOKEN")

# Import Slack Bolt
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler

# LangGraph and LangChain
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict

# Notion client
from notion_client import Client

# Initialize Notion client
notion = Client(auth=NOTION_API_KEY)

# Initialize Gemini LLM (Gemini 1.5 Flash)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

# Verify environment variables are loaded
assert SLACK_BOT_TOKEN, "Missing SLACK_BOT_TOKEN"
assert SLACK_SIGNING_SECRET, "Missing SLACK_SIGNING_SECRET"
assert GOOGLE_API_KEY, "Missing GOOGLE_API_KEY"
assert NOTION_API_KEY, "Missing NOTION_API_KEY"


In [ ]:
class ChatState(TypedDict):
    messages: List[BaseMessage]

def get_all_notion_pages_summary(notion_client: Client, max_pages=5) -> str:
    response = notion_client.search(filter={"property": "object", "value": "page"})
    pages = response.get("results", [])[:max_pages]

    titles = []
    for page in pages:
        props = page.get("properties", {})
        title = None
        if "title" in props and props["title"].get("title"):
            title = props["title"]["title"][0]["plain_text"]
        else:
            # fallback to page id if no title property found
            title = page.get("id")

        if title:
            titles.append(title)

    if not titles:
        return "No Notion pages found in your workspace."

    return "Here are some Notion pages: " + ", ".join(titles)




In [ ]:
summary = get_all_notion_pages_summary(notion, max_pages=2)
print(summary)

In [ ]:
def respond_to_user_with_notion_pages(state: ChatState) -> ChatState:
    messages = state.get("messages", [])
    user_msg = messages[-1] if messages else None

    notion_summary = get_all_notion_pages_summary(notion, max_pages=5)

    prompt = f"{user_msg.content if user_msg else ''}\n\nAdditional context from your Notion workspace:\n{notion_summary}"

    response = llm.invoke([HumanMessage(content=prompt)])

    messages.append(response)
    return {"messages": messages}


In [ ]:
workflow = StateGraph(ChatState)
workflow.add_node("respond", respond_to_user_with_notion_pages)
workflow.set_entry_point("respond")
workflow.set_finish_point("respond")
graph = workflow.compile()


In [ ]:
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)

@app.event("app_mention")
def handle_mention(event, say):
    user_input = event["text"].split(">", 1)[-1].strip()
    user_msg = HumanMessage(content=user_input)

    # Run LangGraph with initial user message
    result = graph.invoke({"messages": [user_msg]})

    bot_reply = result["messages"][-1].content

    say(bot_reply)


In [ ]:
if __name__ == "__main__":
    print("🤖 Slack bot running with Gemini 1.5 Flash + Notion integration...")
    try:
        handler = SocketModeHandler(app, app_token=SOCKET_MODE_APP_TOKEN)
        handler.start()
    except KeyboardInterrupt:
        print("\n🛑 Slack bot stopped manually.")
    except Exception as e:
        print(f"❌ Error starting Slack bot: {e}")


